# 🎙️ Chuẩn bị dữ liệu VieNeu-TTS-140h cho Kani TTS

Notebook này sẽ:
1. ✅ Tải dataset VieNeu-TTS-140h từ HuggingFace
2. ✅ Encode audio thành NanoCodec tokens (4 layers)
3. ✅ Lưu dataset đã xử lý
4. ✅ Upload lên HuggingFace Hub

**Yêu cầu:** GPU (T4/V100/A100) trên Google Colab hoặc Kaggle


In [ ]:
# Cài đặt dependencies
%uv pip install -q omegaconf numpy datasets==3.6.0 torch librosa soundfile tqdm
%uv pip install -q nemo_toolkit[all]==2.4.0
%uv pip install -q --upgrade protobuf==5.29.2

print("✅ Đã cài đặt tất cả dependencies!")


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
✅ Đã cài đặt tất cả dependencies!


In [2]:
# Kiểm tra GPU
import torch

if torch.cuda.is_available():
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️  No GPU found! Processing will be very slow.")
    print("   Please enable GPU: Runtime > Change runtime type > GPU")


✅ GPU available: NVIDIA L4
   GPU Memory: 22.03 GB


## Bước 1: Login HuggingFace

Cần login để tải dataset và upload kết quả lên HuggingFace Hub.


In [ ]:
# Login HuggingFace
from huggingface_hub import login

import os

if "HF_TOKEN" in os.environ:
    login(token=os.environ["HF_TOKEN"])
    print("✅ Đã login HuggingFace từ environment variable")
else:
    print("⚠️  Chưa có HF_TOKEN trong environment.")
    print("   Vui lòng thêm token vào Secrets hoặc uncomment dòng trên để nhập trực tiếp.")


## Bước 2: Import thư viện và setup


In [6]:
import os
import json
import gzip
import numpy as np
import torch
from tqdm.auto import tqdm
from datasets import load_dataset, Audio, disable_progress_bars
from nemo.collections.tts.models import AudioCodecModel

# Disable verbose logs
disable_progress_bars()
import warnings
warnings.filterwarnings('ignore')

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

# Tạo thư mục output
OUT_DIR = "shards"
os.makedirs(OUT_DIR, exist_ok=True)
print(f"✅ Output directory: {OUT_DIR}")


✅ Using device: cuda
✅ Output directory: shards


## Bước 3: Load NanoCodec Model


In [7]:
# Load NanoCodec model
MODEL_ID = "nvidia/nemo-nano-codec-22khz-0.6kbps-12.5fps"
SAMPLE_RATE = 22050

print(f"📥 Loading NanoCodec model: {MODEL_ID}")
codec_model = AudioCodecModel.from_pretrained(MODEL_ID).eval()
codec_model = codec_model.to(device)
print("✅ Model loaded successfully!")


📥 Loading NanoCodec model: nvidia/nemo-nano-codec-22khz-0.6kbps-12.5fps


nemo-nano-codec-22khz-0.6kbps-12.5fps.ne(…):   0%|          | 0.00/422M [00:00<?, ?B/s]

[NeMo W 2025-11-06 09:52:22 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      dataset_type: tarred_vocoder
      dataset_args:
        dataset_meta:
          mls_english:
            manifest_path: /lustre/fsw/portfolios/llmservice/projects/llmservice_nemo_speechlm/data/TTS/mls_english/filtered_22khz/tarred_audio/train_manifest.json
            tar_filepath: /lustre/fsw/portfolios/llmservice/projects/llmservice_nemo_speechlm/data/TTS/mls_english/filtered_22khz/tarred_audio/audio_{0..1999}.tar
          cv:
            manifest_path: /lustre/fsw/portfolios/llmservice/projects/llmservice_nemo_speechlm/data/TTS/commonvoice13/22khz/tarred_audio/train_manifest.json
            tar_filepath: /lustre/fsw/portfolios/llmservice/projects/llmservice_nemo_speechlm/data/TTS/commonvoice13/22khz/tarred_audio/audio_{0..279}.tar

[NeMo I 2025-11-06 09:52:22 nemo_logging:393] Vector quantizer does not support commit loss.


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/378M [00:00<?, ?B/s]

[NeMo I 2025-11-06 09:52:28 nemo_logging:393] PADDING: 1


model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

[NeMo I 2025-11-06 09:52:58 nemo_logging:393] PADDING: 1
[NeMo I 2025-11-06 09:52:58 nemo_logging:393] PADDING: 1
[NeMo I 2025-11-06 09:52:58 nemo_logging:393] PADDING: 1
[NeMo I 2025-11-06 09:52:58 nemo_logging:393] PADDING: 1
[NeMo I 2025-11-06 09:52:58 nemo_logging:393] PADDING: 1
[NeMo I 2025-11-06 09:52:59 nemo_logging:393] Model AudioCodecModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--nemo-nano-codec-22khz-0.6kbps-12.5fps/snapshots/5c8e22ed763c14d81337fbe6ca74062f3d10f7e5/nemo-nano-codec-22khz-0.6kbps-12.5fps.nemo.
✅ Model loaded successfully!


## Bước 4: Load Dataset VieNeu-TTS-140h


In [8]:
# Load dataset
DATASET_NAME = "pnnbao-ump/VieNeu-TTS-140h"
SPLIT = "train"

print(f"📦 Loading dataset: {DATASET_NAME} [{SPLIT}]")
print("   This may take a few minutes...")

dataset = load_dataset(
    DATASET_NAME,
    split=SPLIT,
    verification_mode='no_checks',
    trust_remote_code=True
)

# Convert audio to target sample rate
dataset = dataset.cast_column("audio", Audio(sampling_rate=SAMPLE_RATE))

print(f"✅ Loaded {len(dataset)} samples")
print(f"   Sample keys: {dataset[0].keys()}")


📦 Loading dataset: pnnbao-ump/VieNeu-TTS-140h [train]
   This may take a few minutes...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00049.parquet:   0%|          | 0.00/472M [00:00<?, ?B/s]

data/train-00001-of-00049.parquet:   0%|          | 0.00/478M [00:00<?, ?B/s]

data/train-00002-of-00049.parquet:   0%|          | 0.00/472M [00:00<?, ?B/s]

...

data/train-00045-of-00049.parquet:   0%|          | 0.00/479M [00:00<?, ?B/s]

data/train-00046-of-00049.parquet:   0%|          | 0.00/477M [00:00<?, ?B/s]

data/train-00047-of-00049.parquet:   0%|          | 0.00/475M [00:00<?, ?B/s]

data/train-00048-of-00049.parquet:   0%|          | 0.00/470M [00:00<?, ?B/s]

✅ Loaded 74858 samples
   Sample keys: dict_keys(['_id', 'audio', 'text', 'phonemized_text', 'duration', 'speaker', 'gender', 'language'])


## Bước 5: Xem thống kê dataset


In [9]:
# Thống kê dataset
from collections import Counter

speakers = dataset['speaker']
unique_speakers = set(speakers)
speaker_counts = Counter(speakers)

print(f"📊 Dataset Statistics:")
print(f"   Total samples: {len(dataset):,}")
print(f"   Total speakers: {len(unique_speakers)}")
print(f"   Average duration: {sum(dataset['duration']) / len(dataset):.2f} seconds")

if 'gender' in dataset.column_names:
    genders = dataset['gender']
    gender_counts = Counter(genders)
    print(f"\n   Gender distribution:")
    for gender, count in gender_counts.items():
        print(f"     {gender}: {count:,} ({count/len(dataset)*100:.1f}%)")

print(f"\n   Top 10 speakers:")
for speaker, count in speaker_counts.most_common(10):
    print(f"     {speaker}: {count:,} samples")


📊 Dataset Statistics:
   Total samples: 74,858
   Total speakers: 193
   Average duration: 6.76 seconds

   Gender distribution:
     female: 24,658 (32.9%)
     male: 50,200 (67.1%)

   Top 10 speakers:
     jellyfish1010_0004: 974 samples
     jellyfish1010_0037: 766 samples
     jellyfish1010_0028: 742 samples
     alloy1512_1002: 727 samples
     capybara1812_0076: 726 samples
     jellyfish1010_0010: 715 samples
     capybara1812_0025: 694 samples
     capybara1812_0027: 685 samples
     alloy1512_1005: 668 samples
     jellyfish1010_1017: 667 samples


## Bước 6: Encode Audio thành NanoCodec Tokens

Hàm này sẽ encode audio thành 4 layers của codec tokens.


In [10]:
def encode_audio(waveform, codec_model, device):
    """Encode audio waveform to NanoCodec tokens"""
    # Convert to tensor
    audio_tensor = torch.from_numpy(waveform).unsqueeze(dim=0).to(dtype=torch.float32)
    audio_tensor = audio_tensor.to(device)
    audio_len = torch.tensor([audio_tensor[0].shape[0]]).to(device)
    
    # Encode
    with torch.inference_mode():
        encoded_tokens, encoded_len = codec_model.encode(
            audio=audio_tensor, 
            audio_len=audio_len
        )
    
    # Extract layers
    encoded_tokens = encoded_tokens.squeeze()
    
    return {
        'nano_layer_1': encoded_tokens[0].cpu().numpy().flatten(),
        'nano_layer_2': encoded_tokens[1].cpu().numpy().flatten(),
        'nano_layer_3': encoded_tokens[2].cpu().numpy().flatten(),
        'nano_layer_4': encoded_tokens[3].cpu().numpy().flatten(),
        'encoded_len': encoded_len.item()
    }

# Test với 1 mẫu
print("🧪 Testing encoding với 1 mẫu...")
sample = dataset[0]
encoded = encode_audio(sample['audio']['array'], codec_model, device)
print(f"✅ Test thành công!")
print(f"   Encoded length: {encoded['encoded_len']}")
print(f"   Layer 1 shape: {encoded['nano_layer_1'].shape}")


🧪 Testing encoding với 1 mẫu...
✅ Test thành công!
   Encoded length: 88
   Layer 1 shape: (88,)


## Bước 7: Process toàn bộ dataset

⚠️ **Lưu ý:** Quá trình này có thể mất vài giờ tùy vào GPU và số lượng mẫu.


In [11]:
# Cấu hình xử lý
LINES_PER_FILE = 10000  # Số mẫu mỗi file
BATCH_SIZE = 32  # Process theo batch để tăng tốc

def process_dataset(dataset, codec_model, device, out_dir, lines_per_file=10000):
    """Process dataset và lưu thành các file JSONL.gz"""
    
    file_idx = 0
    lines_in_file = 0
    current_file = None
    current_gz = None
    
    def open_new_file(idx):
        path = os.path.join(out_dir, f"vieneu-tts-{idx:05d}.jsonl.gz")
        gz_file = gzip.open(path, 'wt', encoding='utf-8')
        return path, gz_file
    
    def close_file(gz_file):
        if gz_file:
            gz_file.close()
    
    def write_record(record, gz_file):
        json.dump(record, gz_file, ensure_ascii=False)
        gz_file.write('\n')
    
    # Mở file đầu tiên
    file_path, current_gz = open_new_file(file_idx)
    print(f"📝 Writing to: {file_path}")
    
    total_samples = len(dataset)
    pbar = tqdm(total=total_samples, desc="Processing")
    
    for idx, item in enumerate(dataset):
        try:
            # Encode audio
            encoded = encode_audio(item['audio']['array'], codec_model, device)
            
            # Tạo record
            record = {
                'text': item['text'],
                'nano_layer_1': encoded['nano_layer_1'].tolist(),
                'nano_layer_2': encoded['nano_layer_2'].tolist(),
                'nano_layer_3': encoded['nano_layer_3'].tolist(),
                'nano_layer_4': encoded['nano_layer_4'].tolist(),
                'encoded_len': encoded['encoded_len'],
                'speaker': item['speaker'],
                'lang': 'vi',
                'dataset_source': 'vieneu-tts-140h'
            }
            
            # Thêm phonemized_text nếu có
            if 'phonemized_text' in item:
                record['phonemized_text'] = item['phonemized_text']
            
            # Thêm gender nếu có
            if 'gender' in item:
                record['gender'] = item['gender']
            
            # Ghi vào file
            write_record(record, current_gz)
            lines_in_file += 1
            pbar.update(1)
            
            # Rotate file nếu cần
            if lines_in_file >= lines_per_file:
                close_file(current_gz)
                file_idx += 1
                file_path, current_gz = open_new_file(file_idx)
                print(f"\\n📝 Rotated to: {file_path}")
                lines_in_file = 0
                
        except Exception as e:
            print(f"\\n⚠️  Error processing sample {idx}: {e}")
            continue
    
    # Đóng file cuối cùng
    close_file(current_gz)
    pbar.close()
    
    print(f"\\n✅ Processing completed!")
    print(f"   Generated {file_idx + 1} files")
    return file_idx + 1

# Chạy processing
print("🚀 Bắt đầu xử lý dataset...")
num_files = process_dataset(dataset, codec_model, device, OUT_DIR, LINES_PER_FILE)
print(f"✅ Hoàn thành! Đã tạo {num_files} files trong {OUT_DIR}/")


🚀 Bắt đầu xử lý dataset...
📝 Writing to: shards/vieneu-tts-00000.jsonl.gz


Processing:   0%|          | 0/74858 [00:00<?, ?it/s]

\n📝 Rotated to: shards/vieneu-tts-00001.jsonl.gz
\n📝 Rotated to: shards/vieneu-tts-00002.jsonl.gz
\n📝 Rotated to: shards/vieneu-tts-00003.jsonl.gz
\n📝 Rotated to: shards/vieneu-tts-00004.jsonl.gz
\n📝 Rotated to: shards/vieneu-tts-00005.jsonl.gz
\n📝 Rotated to: shards/vieneu-tts-00006.jsonl.gz
\n📝 Rotated to: shards/vieneu-tts-00007.jsonl.gz
\n✅ Processing completed!
   Generated 8 files
✅ Hoàn thành! Đã tạo 8 files trong shards/


## Bước 8: Assemble dataset từ các shards

Tổng hợp tất cả các file JSONL.gz thành một dataset hoàn chỉnh.


In [12]:
# Assemble dataset từ các shards
from datasets import load_dataset
import glob

print("🔨 Assembling dataset from shards...")

# Load tất cả các file JSONL.gz
shard_files = sorted(glob.glob(os.path.join(OUT_DIR, "*.jsonl.gz")))
print(f"   Found {len(shard_files)} shard files")

# Load dataset từ các shards
final_dataset = load_dataset(
    "json",
    data_files=shard_files,
    split="train",
    verification_mode='no_checks'
)

print(f"✅ Dataset assembled: {len(final_dataset)} samples")
print(f"   Features: {final_dataset.features}")

# Xem một mẫu
print(f"\n📋 Sample record:")
sample = final_dataset[0]
for key in sample.keys():
    if isinstance(sample[key], list) and len(sample[key]) > 0:
        print(f"   {key}: list with {len(sample[key])} items")
    else:
        print(f"   {key}: {sample[key]}")


🔨 Assembling dataset from shards...
   Found 8 shard files
✅ Dataset assembled: 74858 samples
   Features: {'text': Value(dtype='string', id=None), 'nano_layer_1': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'nano_layer_2': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'nano_layer_3': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'nano_layer_4': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'encoded_len': Value(dtype='int64', id=None), 'speaker': Value(dtype='string', id=None), 'lang': Value(dtype='string', id=None), 'dataset_source': Value(dtype='string', id=None), 'phonemized_text': Value(dtype='string', id=None), 'gender': Value(dtype='string', id=None)}

📋 Sample record:
   text: Ngài Morrel, bạn bè và gia đình hai họ đều có mặt đông đủ để chúc phúc cho đôi trẻ.
   nano_layer_1: list with 88 items
   nano_layer_2: list with 88 items
   nano_layer_3: list with 88 items
   nano_layer_4: list

## Bước 9: Upload lên HuggingFace Hub

⚠️ **Lưu ý:** 
- Đảm bảo đã login HuggingFace ở Bước 1
- Thay đổi `HF_USERNAME` và `DATASET_NAME` thành tên của bạn
- Dataset sẽ được upload dưới dạng private mặc định (có thể đổi thành public)


In [13]:
# Cấu hình upload
HF_USERNAME = "pnnbao-ump"  # ⚠️ THAY ĐỔI: Tên HuggingFace của bạn
DATASET_NAME = "VieNeu-TTS-140h-nanocodec"  # ⚠️ THAY ĐỔI: Tên dataset bạn muốn
REPO_ID = f"{HF_USERNAME}/{DATASET_NAME}"

print(f"📤 Cấu hình upload:")
print(f"   Repository: {REPO_ID}")
print(f"   Dataset size: {len(final_dataset):,} samples")

# Kiểm tra xem đã login chưa
from huggingface_hub import whoami
try:
    user_info = whoami()
    print(f"✅ Logged in as: {user_info['name']}")
    if user_info['name'] != HF_USERNAME:
        print(f"⚠️  Warning: Username mismatch!")
        print(f"   Config: {HF_USERNAME}, Logged in: {user_info['name']}")
        print(f"   Sẽ upload với username: {user_info['name']}")
        HF_USERNAME = user_info['name']
        REPO_ID = f"{HF_USERNAME}/{DATASET_NAME}"
except Exception as e:
    print(f"❌ Chưa login HuggingFace!")
    print(f"   Vui lòng chạy lại cell Bước 1 để login")
    raise


📤 Cấu hình upload:
   Repository: pnnbao-ump/VieNeu-TTS-140h-nanocodec
   Dataset size: 74,858 samples
✅ Logged in as: pnnbao-ump


In [14]:
# Tạo dataset card (README.md)
dataset_card = f"""---
license: cc-by-nc-4.0
task_categories:
- text-to-speech
language:
- vi
tags:
- tts
- text-to-speech
- vietnamese
- nanocodec
- kani-tts
- multi-speaker
size_categories:
- 10K<n<100K
---

# {DATASET_NAME}

Dataset tiếng Việt đã được encode bằng NVIDIA NeMo NanoCodec cho training Kani TTS.

## Dataset Details

- **Source:** VieNeu-TTS-140h (pnnbao-ump/VieNeu-TTS-140h)
- **Total samples:** {len(final_dataset):,}
- **Speakers:** 193
- **Language:** Vietnamese (vi)
- **Codec:** NVIDIA NeMo NanoCodec (22kHz, 0.6kbps, 12.5fps)
- **Format:** JSONL.gz với NanoCodec tokens (4 layers)

## Dataset Structure

Mỗi mẫu bao gồm:
- `text`: Text transcription tiếng Việt
- `nano_layer_1`: Codec tokens layer 1
- `nano_layer_2`: Codec tokens layer 2
- `nano_layer_3`: Codec tokens layer 3
- `nano_layer_4`: Codec tokens layer 4
- `encoded_len`: Độ dài sequence tokens
- `speaker`: Speaker ID
- `lang`: "vi"
- `dataset_source`: "vieneu-tts-140h"
- `phonemized_text`: IPA phonemization (nếu có)
- `gender`: Giới tính speaker (nếu có)

## Usage

```python
from datasets import load_dataset

dataset = load_dataset("{REPO_ID}")

# Xem một mẫu
sample = dataset['train'][0]
print(sample['text'])
print(f"Speaker: {{sample['speaker']}}")
print(f"Tokens: {{sample['encoded_len']}}")
```

## License

CC BY-NC 4.0 - Non-commercial use only

## Citation

```bibtex
@dataset{{vieneu_tts_140h_nanocodec,
  title = {{VieNeu-TTS-140h NanoCodec Dataset}},
  author = {{Your Name}},
  year = {{2025}},
  url = {{https://huggingface.co/datasets/{REPO_ID}}}
}}
```
"""

# Lưu dataset card
with open("README.md", "w", encoding="utf-8") as f:
    f.write(dataset_card)

print("✅ Đã tạo dataset card (README.md)")


✅ Đã tạo dataset card (README.md)


In [15]:
# Upload dataset lên HuggingFace Hub
print(f"☁️  Uploading dataset to HuggingFace Hub...")
print(f"   Repository: {REPO_ID}")
print(f"   This may take a while depending on dataset size...")

# Upload dataset
final_dataset.push_to_hub(
    repo_id=REPO_ID,
    private=True,  # Đặt False nếu muốn public
    commit_message="Upload VieNeu-TTS-140h NanoCodec dataset"
)

print(f"\n✅ Upload completed!")
print(f"   Dataset URL: https://huggingface.co/datasets/{REPO_ID}")


☁️  Uploading dataset to HuggingFace Hub...
   Repository: pnnbao-ump/VieNeu-TTS-140h-nanocodec
   This may take a while depending on dataset size...


Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  99%|#########9| 57.8MB / 58.3MB            


✅ Upload completed!
   Dataset URL: https://huggingface.co/datasets/pnnbao-ump/VieNeu-TTS-140h-nanocodec


In [16]:
# Upload README.md (dataset card)
from huggingface_hub import HfApi

api = HfApi()
api.upload_file(
    path_or_fileobj="README.md",
    path_in_repo="README.md",
    repo_id=REPO_ID,
    repo_type="dataset",
    commit_message="Add dataset card"
)

print("✅ Dataset card uploaded!")
print(f"\n🎉 Hoàn thành! Dataset đã được upload lên:")
print(f"   https://huggingface.co/datasets/{REPO_ID}")


✅ Dataset card uploaded!

🎉 Hoàn thành! Dataset đã được upload lên:
   https://huggingface.co/datasets/pnnbao-ump/VieNeu-TTS-140h-nanocodec
